# Traffic Sign Classification with Tensorflow


## Step 0: Load Image Dataset

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from zipfile import ZipFile

import pickle
import os
import cv2
from tqdm import tqdm

# Define variables
setnames = []
data     = dict()
X        = dict()
y        = dict()

path     = "/home/tiwi/Documents/mldata/"
zipfiles = "traffic-signs-data.zip"
file_concat    = lambda path, zipfiles, filename: path + zipfiles.rpartition('.')[0] + '/' + filename
file_body_of   = lambda filename: os.path.split(filename)[1].rpartition('.')[0] 

print("Filenames in data container: \n")
for filename in ZipFile(path+zipfiles).namelist():
    print(filename)
    # Create setnames from filenames: e.g. "train", "test", ...
    setnames.append(file_body_of(filename))
    # Get full path + file for each pickle file
    with open(file_concat(path, zipfiles, filename), mode='rb') as f:
        data[setnames[-1]] = pickle.load(f)
        
for setname in setnames:
    X[setname], y[setname] = data[setname]['features'], data[setname]['labels']
        


## Step 1: Print Dataset Information

In [ ]:
classes = pd.read_csv("../CarND-Traffic-Sign-Classifier-Project/signnames.csv")
nb_classes = len(set(y["train"]))

print("Number of training examples:\t", X["train"].shape[0])
print("Number of testing examples:\t",  X["test"].shape[0])
print("Image data shape:\t\t",          X["train"].shape[1:])
print("Number of classes:\t\t",         nb_classes)

print(classes)

In [ ]:
def plot_traffic_signs(X, rnd_seed=1):
    
    # Define square number of example images
    n = np.int(np.floor(np.sqrt(nb_classes)))
    nb_plots = n**2
    np.random.seed(rnd_seed)
    random_images = [X["train"][i,::] for i in np.random.randint(0, X["train"].shape[0], nb_plots)]

    # Setup a grid [n x n] grid of subplots
    fig, axes = plt.subplots(n, n, figsize=(12, 12), subplot_kw={'xticks': [], 'yticks': []})
    fig.subplots_adjust(hspace=0.0, wspace=0.0)

    for img, ax, img_class in zip(random_images, axes.flat, classes["SignName"][0:nb_plots]):
        if img.shape[-1] < 3:
            ax.imshow(img.squeeze())
        else:
            ax.imshow(img)

        
    plt.show()
plot_traffic_signs(X)

## Step 2: Design and Test a Model Architecture
The basic idea is to implement a simple CNN archictecture that gets a reasonable accurancy. 

### Preprocessing 1: Normalization

In [ ]:
def normalize_brightness(features):
    equ = dict()
    for key in features: 
        sx, sy = features[key].shape[1], features[key].shape[2] 
        equ[key] = np.zeros((features[key].shape[0:3]))
        for idx, img in enumerate(features[key]):
            tmp_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            tmp_img = cv2.equalizeHist(tmp_img)
            equ[key][idx,:,:] = tmp_img
        equ[key] = equ[key].reshape(-1, sx, sy, 1)
        print(equ[key].shape)
        
    return equ

In [ ]:

print("Before Normalization")
print("Shape:\t", X["train"].shape)
print("Min:\t", X["train"].min())
print("Max:\t", X["train"].max())
print("Mean:\t", np.mean(X["train"]))

print("Shape y:", y["train"].shape)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, OneHotEncoder

def normalize(features, method='standard'):
    X = features.copy()
    if method=='standard':
        scaler = StandardScaler()
    elif method=='norm':
        scaler = Normalizer()
    elif method=='minmax':
        scaler = MinMaxScaler(feature_range=(0, 1))
        
    for key in X:
        shape      = X[key].shape
        image_size = shape[1] * shape[2] * shape[3]    
        X[key]     = X[key].reshape(-1, image_size)
        X[key]     = scaler.fit_transform(np.float32(X[key])).reshape(-1, shape[1], shape[2], shape[3])
    return X

def encode(labels):
    y = labels.copy()
    nb_classes = len(set(y["train"]))
    encoder = OneHotEncoder(sparse=False)

    for key in y:
        y[key] = encoder.fit_transform(y[key].reshape(-1, 1))
    return y



def get_batches(features, labels, batch_size):
    nb_samples = len(features)
    split_at = np.arange(batch_size, nb_samples, batch_size)
    
    features, labels = shuffle(features, labels)
    
    feature_batches = np.split(features, split_at)
    label_batches   = np.split(labels, split_at)
    
    return feature_batches, label_batches

In [ ]:
X = normalize_brightness(X)
X = normalize(X, method='minmax')
Y = encode(y)

print("\nAfter Normalization")
print("Shape:\t", X["train"].shape)
print("Min:\t", X["train"].min())
print("Max:\t", X["train"].max())
print("Mean:\t", np.mean(X["train"]))

print("Shape y:", y["train"].shape)

print("Shape Y:", Y["train"].shape)
print("Type Y:", type(Y["train"]))

assert type(Y['train']) == type(X['train'])
assert len(Y['train']) == len(y['train'])

plot_traffic_signs(X)

### Model Architecture Design

In [ ]:
def dense(x, nb_nodes, scope):
    return tf.contrib.layers.fully_connected(x, nb_nodes, activation_fn=None, scope=scope)

def dense_bn(x, nb_nodes, phase, scope):
    with tf.variable_scope(scope):
        h1 = tf.contrib.layers.fully_connected(x, nb_nodes, 
                                               activation_fn=None,
                                               scope='dense')
        h2 = tf.contrib.layers.batch_norm(h1, 
                                          center=True, scale=True, 
                                          is_training=phase,
                                          scope='bn')
        return tf.nn.relu(h2, 'relu')
    
def conv_bn(x, nb_filters, kernel_size, phase, activation='relu', stride=1, padding='SAME', scope='conv_nb'):
    with tf.variable_scope(scope):
        h1 = tf.contrib.layers.conv2d(x, nb_filters, kernel_size,
                                      stride=stride, padding=padding,
                                      activation_fn=None, scope='conv') 
        h2 = tf.contrib.layers.batch_norm(h1, 
                                          center=True, scale=True, 
                                          is_training=phase,
                                          scope='bn')
        if activation == None:
            return h2
        elif activation == 'relu':
            return tf.nn.relu(h2, 'relu')
    
def residuum_stack(x, nb_filters, kernel_size, phase, scope='res_stack'):
    
    input_size  = x.get_shape()[3]
    output_size = nb_filters
    print("Sizes: ", input_size, output_size)

    with tf.variable_scope(scope):
        with tf.variable_scope('A'):
            h = conv_bn(x, nb_filters, kernel_size, phase, activation='relu', scope='conv_bn')
        with tf.variable_scope('B'):
            h = conv_bn(h, nb_filters, kernel_size, phase, activation='relu', scope='conv_bn')

        if input_size == output_size:
            shortcut = x
            print("in==out:", shortcut.get_shape()[3], h.get_shape()[3])
            return tf.nn.relu(h + shortcut, 'relu')

        else:
            shortcut = tf.contrib.layers.conv2d(x,
                                            output_size,
                                            kernel_size=[1,1],
                                            activation_fn=None,
                                            scope='conv_1x1')
            if input_size > output_size:
                print("in>out:", x.get_shape()[3], h.get_shape()[3])
            elif input_size < output_size:
                print("in<out:", x.get_shape()[3], h.get_shape()[3])

            return tf.nn.relu(h + shortcut, 'relu')
        
def inception_A(x, phase, scope='block_A'):
    sc = lambda name, n: name + '_' + str(n) + 'x' + str(n)
    nb_filters_1 = 64
    nb_filters_2 = 96
    with tf.variable_scope(scope):
        with tf.variable_scope('branch_0'):            
            shortcut = conv_bn(x, nb_filters_2, [1,1], phase, scope='shortcut')

        with tf.variable_scope('branch_1'):
            h1 = conv_bn(x, nb_filters_2, [1,1], phase, scope='conv_1_1x1')
            
        with tf.variable_scope('branch_2'):
            h2a = conv_bn(x, nb_filters_1, [1,1], phase, scope='conv_2a_1x1')
            h2 = conv_bn(h2a, nb_filters_2, [3,3], phase, scope='conv_2b_3x3')
            
        with tf.variable_scope('branch_3'):
            h3a = conv_bn(x, nb_filters_1, [1,1], phase, scope='conv_3a_1x1') 
            h3b = conv_bn(h3a, nb_filters_2, [3,3], phase, scope='conv_3b_3x3') 
            h3  = conv_bn(h3b, nb_filters_2, [3,3], phase, scope='conv_3c_3x3')
        return tf.concat([shortcut, h1, h2, h3], 3, 'concat')
            
def inception_B(x, phase, scope='block_B'):

    nb_filters_1 = 16  #128
    nb_filters_2 = 32  #192
    nb_filters_3 = 48 #224
    nb_filters_4 = 64 #256
    nb_filters_5 = 72 #384

    with tf.variable_scope(scope):
        with tf.variable_scope('branch_0'):            
            shortcut = conv_bn(x, nb_filters_5, [1,1], phase, scope='shortcut')

        with tf.variable_scope('branch_1'):
            h1a = conv_bn(x,   nb_filters_2, [1,1], phase, scope='conv_1a_1x1')
            h1b = conv_bn(h1a, nb_filters_3, [1,7], phase, scope='conv_1b_1x7')
            h1  = conv_bn(h1b, nb_filters_4, [7,1], phase, scope='conv_1c_7x1')
            
        with tf.variable_scope('branch_2'):
            h2a = conv_bn(x,   nb_filters_2, [1,1], phase, scope='conv_2a_1x1')
            h2b = conv_bn(h2a, nb_filters_2, [1,7], phase, scope='conv_2b_1x7')
            h2c = conv_bn(h2b, nb_filters_3, [7,1], phase, scope='conv_2c_7x1')
            h2d = conv_bn(h2c, nb_filters_3, [1,7], phase, scope='conv_2d_1x7')
            h2  = conv_bn(h2d, nb_filters_4, [7,1], phase, scope='conv_2e_7x1')
            
        return tf.concat([shortcut, h1, h2], 3, 'concat')
        
def inception_C():
    pass

In [ ]:
tf.reset_default_graph()
x         = tf.placeholder('float32', (None, 32, 32, 1), name='x')
y_one_hot = tf.placeholder('float32', (None, nb_classes), name='y_one_hot')
phase     = tf.placeholder(tf.bool, name='phase')

h01 = conv_bn(x,   32,  [3, 3], phase, stride=1, scope='layer01')
h02 = conv_bn(h01, 64, [3, 3], phase, stride=1, scope='layer02')

h03 = residuum_stack(h02, 128,  [3, 3], phase, scope='layer03')
h04 = inception_A(h03, phase, 'block_A')
h05 = conv_bn(h04, 64, [3, 3], phase, stride=2, scope='layer05')
h06 = inception_B(h05, phase, 'block_B')
h0e = residuum_stack(h03, 128, [3, 3], phase, scope='layer0e')

h00 = tf.contrib.layers.flatten(h0e, scope='flatten')
h1  = dense_bn(h00, 96, phase, scope='layer1')
logits = dense(h1, nb_classes, scope='logits')

with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(
            tf.equal(tf.argmax(y_one_hot, 1), tf.argmax(logits, 1)),
            'float32'))

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_one_hot))
   

In [ ]:
def train(nb_epochs, X_batches, Y_batches, learning_rate):
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        # Ensures that we execute the update_ops before performing the train_step
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())

        history = []
        for epoch in range(0, nb_epochs):
            X_batches, Y_batches = shuffle(X_batches, Y_batches)

            for X_batch, Y_batch in zip(X_batches, Y_batches):
                session.run(train_step,
                             feed_dict={'x:0': X_batch, 
                                        'y_one_hot:0': Y_batch, 
                                        'phase:0': 1})
            tr = session.run([loss, accuracy], 
                              feed_dict={'x:0': X_batch,
                                         'y_one_hot:0': Y_batch,
                                         'phase:0': 1})
            t = session.run([loss, accuracy], 
                         feed_dict={'x:0': X_batch,
                                    'y_one_hot:0': Y_batch,
                                    'phase:0': 0})
            history += [[epoch] + tr + t]
            print("EPOCH {0} {1}".format(epoch+1, history[-1]))

#            print(history[-1])
    return history

In [ ]:
nb_epochs = 2
batch_size = 128
nb_samples = 1 * batch_size
lr = 0.1
X_batches, Y_batches = get_batches(X['train'][0:nb_samples,::],
                                   Y['train'][0:nb_samples,::],
                                   batch_size=batch_size)

history = train(nb_epochs, X_batches, Y_batches, learning_rate=lr)

In [ ]:
aa = []
for h in history:
    aa.append(h[1])
plt.plot(aa)
plt.show()

In [ ]:
1/43

In [ ]:
aaa = range(1,1)
for a in aaa:
    print(a)

In [ ]:
whos

### Train the Model

In [ ]:
tf.reset_default_graph()
conv_params = dict({'kernel_size': [], 'nb_kernels':[]})
conv_params['kernel_size'] = [[5, 5],
                              [3, 3],
                              [3, 1],
                              [1, 3],
                              [5, 5]]
conv_params['nb_kernels'] = [6,
                             6,
                             12,
                             12,
                             32]    

x         = tf.placeholder(tf.float32, shape=(None, 32, 32, 1))
one_hot_y = tf.placeholder(tf.float32, shape=(None, nb_classes))

logits = create_model(x, params=conv_params, nb_classes=nb_classes, scope_reuse=False)


cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y)
loss = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=0.02).minimize(loss)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def evaluate(X_batches, Y_batches, accuracy_operation):
    
    total_accuracy   = 0
    nb_total_samples = 0
    sess = tf.get_default_session()
    
    for X_batch, Y_batch in zip(X_batches, Y_batches):
        accuracy = sess.run(accuracy_operation, feed_dict={x: X_batch, one_hot_y: Y_batch})
        
        nb_samples        = len(Y_batch)
        total_accuracy   += (accuracy * nb_samples)
        nb_total_samples += nb_samples

    return total_accuracy / nb_total_samples


In [ ]:
nb_epochs = 1
nb_samples = 300
batch_size = 15
losses = []
validation_accuracy = []
X_batches, Y_batches = get_batches(X['train'][0:nb_samples,::],
                                   Y['train'][0:nb_samples,::],
                                   batch_size=batch_size)


for X_batch, Y_batch in zip(X_batches, Y_batches):
    assert(X_batch.shape[0]==batch_size)
    assert(Y_batch.shape[0]==batch_size)
    
# Add ops to save and restore all the variables.
saver = tf.train.Saver()
checkpoint_file = "./checkpoints/model.ckpt"
checkpoint_path = ""

with tf.Session() as session:
    # Restore variables from disk.
    if os.path.isfile(checkpoint_file + ".meta"):
        saver.restore(session, tf.train.latest_checkpoint('./checkpoints/'))
        print("Model restored.")
    else:
        session.run(tf.global_variables_initializer())
    for epoch in range(nb_epochs):
        print("EPOCH {} ...".format(epoch+1))
        X_batches, Y_batches = shuffle(X_batches, Y_batches)
        for X_batch, Y_batch in zip(X_batches, Y_batches):
            assert(X_batch.shape[0]==batch_size)
            assert(Y_batch.shape[0]==batch_size)
            X_batch, Y_batch = shuffle(X_batch, Y_batch)
            _,l = session.run([optimizer, loss], feed_dict={x:X_batch, one_hot_y:Y_batch})
            losses.append(l)
            print("Loss = {:.3f}".format(l))

        acc = evaluate(X_batches[0:2], Y_batches[0:2], accuracy_operation)
        validation_accuracy.append(acc)
        print("Validation Accuracy = {:.3f}".format(acc))
        # Save the variables to disk.
        checkpoint_path = saver.save(session, checkpoint_file)

In [ ]:
print(checkpoint_file)
print(checkpoint_path)

print(os.path.exists(checkpoint_file + '*'))
os.path.exists('./checkpoint')


In [ ]:
plt.plot(losses)
plt.show()
print(losses)

In [ ]:
XX = dict()
for setname in setnames:
    XX[setname] = data[setname]['features']
   

res = normalize_brightness(XX)

print(res["train"].shape)

### ToDos
* add brightness normalization
* augement dataset to get balanced labels distribution
* run full dataset on AWS
* start with tensorboard


In [ ]:
(X["train"].shape[0:3].append(1))

tf.reset_default_graph()
x         = tf.placeholder('float32', (None, 32, 32, 1), name='x')
y_one_hot = tf.placeholder('float32', (None, nb_classes), name='y_one_hot')
phase     = tf.placeholder(tf.bool, name='phase')

h01 = conv_bn(x,  128, [3, 3], phase, scope='layer01')
h02 = conv_bn(h01, 64, [3, 3], phase, scope='layer02')
h03 = conv_bn(h02, 32, [3, 3], phase, scope='layer03')
h04 = conv_bn(h03, 16, [3, 3], phase, scope='layer04')
h05 = conv_bn(h04, 8,  [3, 3], phase, scope='layer05')
h0e = conv_bn(h05, 4,  [3, 3], phase, scope='layer0e')

h00 = tf.contrib.layers.flatten(h0e, scope='flatten')
h1  = dense_bn(h00, 96, phase, scope='layer1')
logits = dense(h1, nb_classes, scope='logits')

with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(
            tf.equal(tf.argmax(y_one_hot, 1), tf.argmax(logits, 1)),
            'float32'))

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_one_hot))
    
    